# RE RE RE RE FEATURE ENGINEERING OF DATA - ptp on purchase date - new

In [1]:
import numpy as np
import pandas as panda
from dateutil import relativedelta
import datetime

In [2]:
train_data_path = 'all_new/train_2.csv'
test_data_path = 'all_new/test_2.csv'
new_transaction_path = 'all_new/new_merchant_transactions.csv'
historical_transaction_path = 'all_new/historical_transactions.csv'
new_treated_transaction_path = 'all_new/treated_new_transaction.csv'
treated_historical_transaction_path = 'all_new/treated_historical_transactions.csv'



In [3]:
## deres nothing else we can do now...with this data
## lets look at new transactions first , since it is a smaller data sets

new_transaction_data = panda.read_csv(new_transaction_path)
new_transaction_data_types = new_transaction_data.dtypes
new_transaction_column_types = {}
for index, column_name in enumerate(new_transaction_data.columns.tolist()):
    new_transaction_column_types[column_name] = new_transaction_data_types[index]
del new_transaction_data
new_transaction_data = panda.read_csv(new_transaction_path, dtype = new_transaction_column_types)
new_transaction_data.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [4]:
def prep_for_aggregate(table):
    
    table['authorized_flag'] = table.authorized_flag.map({'Y':1,'N':0})
    table['category_1'] = table.category_1.map({'Y':1, 'N':0})
    most_appearing = table.category_2.value_counts().idxmax()
    table.category_2.fillna(value= most_appearing, inplace=True)
    table['enc_category_2_1'] = (table.category_2==1) + 0
    table['enc_category_2_2'] = (table.category_2==2) + 0
    table['enc_category_2_3'] = (table.category_2==3) + 0
    table['enc_category_2_4'] = (table.category_2==4) + 0
    table['enc_category_2_5'] = (table.category_2==5) + 0

    table.drop(columns = ['category_2'], inplace=True)
    most_appearing = table.category_3.value_counts().idxmax()
    table.category_3.fillna(value = most_appearing, inplace = True)
    table['enc_category_3_A'] = (table.category_3=='A') +0
    table['enc_category_3_B'] = (table.category_3=='B') +0
    table['enc_category_3_C'] = (table.category_3=='C') +0
    table.drop(columns = ['category_3'], inplace = True)
    table['enc_purchase_date'] = panda.DatetimeIndex(table.purchase_date).astype(np.int64)*1e-9
    table.drop(columns = ['purchase_date'], inplace = True)
    most_appearing = table.city_id.value_counts().idxmax()
    table.city_id.replace({-1:most_appearing}, inplace = True)
    most_appearing = table.state_id.value_counts().idxmax()
    table.state_id.replace({-1:most_appearing}, inplace = True)
    most_appearing = table.subsector_id.value_counts().idxmax()
    table.subsector_id.replace({-1:most_appearing}, inplace = True)
    
    

In [5]:
prep_for_aggregate(new_transaction_data)
new_transaction_data.head()

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,state_id,subsector_id,enc_category_2_1,enc_category_2_2,enc_category_2_3,enc_category_2_4,enc_category_2_5,enc_category_3_A,enc_category_3_B,enc_category_3_C,enc_purchase_date
0,1,C_ID_415bb3a509,107,0,1,307,M_ID_b0c793002c,1,-0.557574,9,19,1,0,0,0,0,0,1,0,1.520780e+09
1,1,C_ID_415bb3a509,140,0,1,307,M_ID_88920c89e8,1,-0.569580,9,19,1,0,0,0,0,0,1,0,1.521486e+09
2,1,C_ID_415bb3a509,330,0,1,507,M_ID_ad5237ef6b,2,-0.551037,9,14,1,0,0,0,0,0,1,0,1.524752e+09
3,1,C_ID_415bb3a509,69,1,1,661,M_ID_9e84cda3b1,1,-0.671925,9,8,1,0,0,0,0,0,1,0,1.520416e+09
4,1,C_ID_ef55cf8d4b,69,1,1,166,M_ID_3c86fa3831,1,-0.659904,9,29,1,0,0,0,0,0,1,0,1.521753e+09


In [6]:
## we will take the 
## sum,mean,max of purchase amount, 
## sum,mean,max of month lag by card_id
## mean of authorized_flag,
## num of authorized ==1
## num of authorized ==0
## sum, mean, max of installments
## mean of category_1
## sum of enc_category_2* and enc_category_3*

def count_authorized_one(x):
    return x.eq(1).sum()
def count_authorized_zero(x):
    return x.eq(0).sum()
def count_category_1_one(x):
    return x.eq(1).sum()
def count_category_1_zero(x):
    return x.eq(0).sum()

def count_of_unique_cities(x):
    return x.nunique()

def count_of_unique_subsector_id(x):
    return x.nunique()

def count_of_unique_state_id(x):
    return x.nunique()


In [7]:

# aggregate = {
    
# 'purchase_amount' : {'new_purchase_amount_sum':'sum','new_purchase_amount_mean': 'mean','new_purchase_amount_max':'max'},
# 'month_lag' : {'new_month_lag_sum':'sum', 'new_month_lag_mean':'mean','new_month_lag_max':'max'} ,
# 'enc_category_2_1':{'new_category_2_1_sum':'sum'},
# 'enc_category_2_2':{'new_category_2_2_sum':'sum'},
# 'enc_category_2_3':{'new_category_2_3_sum':'sum'},
# 'enc_category_2_4':{'new_category_2_4_sum':'sum'},
# 'enc_category_2_5':{'new_category_2_5_sum':'sum'},    
# 'enc_category_3_A':{'new_category_3_A_sum':'sum'},
# 'enc_category_3_B':{'new_category_3_B_sum':'sum'},
# 'enc_category_3_C':{'new_category_3_C_sum':'sum'},
# 'installments': {'new_installments_sum':'sum','new_installments_mean':'mean','new_installments_max':'max'},
# 'authorized_flag': {'new_authorized_flag_mean':'mean', 'new_authorized_flag_count_authorized_one':count_authorized_one, 'new_authorized_flag_count_authorized_zero':count_authorized_zero},
# 'category_1' :{'new_category_1_mean':'mean', 'new_category_1_count_category_1_one':count_category_1_one, 'new_category_1_count_category_1_zero':count_category_1_zero},
# 'enc_purchase_date' : {'new_purchase_date_mean':'mean','new_purchase_date_min':'min','new_purchase_date_max':'max'},
# 'city_id' : {'new_city_id_nunique':count_of_unique_cities},
# 'state_id': {'new_state_id_nunique':count_of_unique_state_id},
# 'subsector_id' : {'new_subsector_id_nunique':count_of_unique_subsector_id}
    
# }



aggregate = {
    
'purchase_amount' : ['sum','mean','max'],
'month_lag' : ['sum','mean','max'] ,
'enc_category_2_1':['sum'],
'enc_category_2_2': ['sum'],
'enc_category_2_3': ['sum'],
'enc_category_2_4': ['sum'],
'enc_category_2_5': ['sum'],    
'enc_category_3_A': ['sum'],
'enc_category_3_B': ['sum'],
'enc_category_3_C': ['sum'],
'installments': ['sum','mean','max'],
'authorized_flag': ['mean', count_authorized_one, count_authorized_zero],
'category_1' :['mean', count_category_1_one, count_category_1_zero],
'enc_purchase_date' : ['mean','min','max',np.ptp],
'city_id' : [count_of_unique_cities],
'state_id': [count_of_unique_state_id],
'subsector_id' : [count_of_unique_subsector_id]
    
}


In [8]:
group_by = new_transaction_data.groupby(['card_id'])

In [9]:
len(new_transaction_data)

1963031

In [10]:
t = group_by.agg(aggregate)

In [11]:
t.head()

purchase_amount                     month_lag                \
                            sum      mean       max       sum      mean max   
card_id                                                                       
C_ID_00007093c1       -1.328524 -0.664262 -0.656749         4  2.000000   2   
C_ID_0001238066      -14.850055 -0.571156 -0.078318        35  1.346154   2   
C_ID_0001506ef0       -1.447354 -0.723677 -0.715352         2  1.000000   1   
C_ID_0001793786       -0.229620 -0.007407  3.129932        41  1.322581   2   
C_ID_000183fdda       -6.590778 -0.599162 -0.107680        14  1.272727   2   

                enc_category_2_1 enc_category_2_2 enc_category_2_3  \
                             sum              sum              sum   
card_id                                                              
C_ID_00007093c1                1                0                1   
C_ID_0001238066               23                0                0   
C_ID_0001506ef0                0                0                2   
C_ID_0001793786               17                8                5   
C_ID_000183fdda                0                0               11   

                enc_category_2_4             ...              category_1  \
                             sum             ...                    mean   
card_id                                      ...                           
C_ID_00007093c1                0             ...                0.000000   
C_ID_0001238066                0             ...                0.076923   
C_ID_0001506ef0                0             ...                0.000000   
C_ID_0001793786                0             ...                0.000000   
C_ID_000183fdda                0             ...                0.000000   

                                                           enc_purchase_date  \
                count_category_1_one count_category_1_zero              mean   
card_id                                                                        
C_ID_00007093c1                    0                     2      1.523023e+09   
C_ID_0001238066                    2                    24      1.522335e+09   
C_ID_0001506ef0                    0                     2      1.521474e+09   
C_ID_0001793786                    0                    31      1.512327e+09   
C_ID_000183fdda                    0                    11      1.521578e+09   

                                                                      city_id  \
                          min           max        ptp count_of_unique_cities   
card_id                                                                         
C_ID_00007093c1  1.522754e+09  1.523291e+09   537024.0                      2   
C_ID_0001238066  1.519923e+09  1.525118e+09  5195343.0                      7   
C_ID_0001506ef0  1.521239e+09  1.521710e+09   471152.0                      1   
C_ID_0001793786  1.510761e+09  1.514742e+09  3981096.0                      7   
C_ID_000183fdda  1.519994e+09  1.525100e+09  5106807.0                      2   

                                state_id                 subsector_id  
                count_of_unique_state_id count_of_unique_subsector_id  
card_id                                                                
C_ID_00007093c1                        2                            2  
C_ID_0001238066                        3                            9  
C_ID_0001506ef0                        1                            2  
C_ID_0001793786                        4                           14  
C_ID_000183fdda                        2                            6  

[5 rows x 30 columns]

In [12]:
t_cols=['card_id']
t_cols.extend(['_'.join(i) for i in t.columns.tolist()])

t_cols

['card_id',
 'purchase_amount_sum',
 'purchase_amount_mean',
 'purchase_amount_max',
 'month_lag_sum',
 'month_lag_mean',
 'month_lag_max',
 'enc_category_2_1_sum',
 'enc_category_2_2_sum',
 'enc_category_2_3_sum',
 'enc_category_2_4_sum',
 'enc_category_2_5_sum',
 'enc_category_3_A_sum',
 'enc_category_3_B_sum',
 'enc_category_3_C_sum',
 'installments_sum',
 'installments_mean',
 'installments_max',
 'authorized_flag_mean',
 'authorized_flag_count_authorized_one',
 'authorized_flag_count_authorized_zero',
 'category_1_mean',
 'category_1_count_category_1_one',
 'category_1_count_category_1_zero',
 'enc_purchase_date_mean',
 'enc_purchase_date_min',
 'enc_purchase_date_max',
 'enc_purchase_date_ptp',
 'city_id_count_of_unique_cities',
 'state_id_count_of_unique_state_id',
 'subsector_id_count_of_unique_subsector_id']

In [13]:
t.reset_index(inplace=True)
t.columns = ['new_'+ i for i in t_cols]
t.head()

,new_card_id,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_max,new_month_lag_sum,new_month_lag_mean,new_month_lag_max,new_enc_category_2_1_sum,new_enc_category_2_2_sum,new_enc_category_2_3_sum,...,new_category_1_mean,new_category_1_count_category_1_one,new_category_1_count_category_1_zero,new_enc_purchase_date_mean,new_enc_purchase_date_min,new_enc_purchase_date_max,new_enc_purchase_date_ptp,new_city_id_count_of_unique_cities,new_state_id_count_of_unique_state_id,new_subsector_id_count_of_unique_subsector_id
0,C_ID_00007093c1,-1.328524,-0.664262,-0.656749,4,2.000000,2,1,0,1,...,0.000000,0,2,1.523023e+09,1.522754e+09,1.523291e+09,537024.0,2,2,2
1,C_ID_0001238066,-14.850055,-0.571156,-0.078318,35,1.346154,2,23,0,0,...,0.076923,2,24,1.522335e+09,1.519923e+09,1.525118e+09,5195343.0,7,3,9
2,C_ID_0001506ef0,-1.447354,-0.723677,-0.715352,2,1.000000,1,0,0,2,...,0.000000,0,2,1.521474e+09,1.521239e+09,1.521710e+09,471152.0,1,1,2
3,C_ID_0001793786,-0.229620,-0.007407,3.129932,41,1.322581,2,17,8,5,...,0.000000,0,31,1.512327e+09,1.510761e+09,1.514742e+09,3981096.0,7,4,14
4,C_ID_000183fdda,-6.590778,-0.599162,-0.107680,14,1.272727,2,0,0,11,...,0.000000,0,11,1.521578e+09,1.519994e+09,1.525100e+09,5106807.0,2,2,6


In [14]:
t.rename(columns = {'new_card_id':'card_id'}, inplace=True)
t.head()

,card_id,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_max,new_month_lag_sum,new_month_lag_mean,new_month_lag_max,new_enc_category_2_1_sum,new_enc_category_2_2_sum,new_enc_category_2_3_sum,...,new_category_1_mean,new_category_1_count_category_1_one,new_category_1_count_category_1_zero,new_enc_purchase_date_mean,new_enc_purchase_date_min,new_enc_purchase_date_max,new_enc_purchase_date_ptp,new_city_id_count_of_unique_cities,new_state_id_count_of_unique_state_id,new_subsector_id_count_of_unique_subsector_id
0,C_ID_00007093c1,-1.328524,-0.664262,-0.656749,4,2.000000,2,1,0,1,...,0.000000,0,2,1.523023e+09,1.522754e+09,1.523291e+09,537024.0,2,2,2
1,C_ID_0001238066,-14.850055,-0.571156,-0.078318,35,1.346154,2,23,0,0,...,0.076923,2,24,1.522335e+09,1.519923e+09,1.525118e+09,5195343.0,7,3,9
2,C_ID_0001506ef0,-1.447354,-0.723677,-0.715352,2,1.000000,1,0,0,2,...,0.000000,0,2,1.521474e+09,1.521239e+09,1.521710e+09,471152.0,1,1,2
3,C_ID_0001793786,-0.229620,-0.007407,3.129932,41,1.322581,2,17,8,5,...,0.000000,0,31,1.512327e+09,1.510761e+09,1.514742e+09,3981096.0,7,4,14
4,C_ID_000183fdda,-6.590778,-0.599162,-0.107680,14,1.272727,2,0,0,11,...,0.000000,0,11,1.521578e+09,1.519994e+09,1.525100e+09,5106807.0,2,2,6


In [15]:
len(t)

290001

In [16]:
t.to_csv('all_new/agg_new_1.csv', index=False)